# ⚡ Intermediaire | ⏱ 45 min | 🔑 Concepts : __init__, __del__, __new__, cycle de vie

# Constructeurs et Destructeurs en Python

## Objectifs

À la fin de ce notebook, vous serez capable de :
- Comprendre la différence entre `__init__` (initialiseur) et `__new__` (constructeur)
- Utiliser `__del__` comme destructeur/finaliseur
- Comprendre le cycle de vie complet d'un objet Python
- Maîtriser le garbage collector et le comptage de références
- Utiliser des paramètres par défaut et valider dans `__init__`
- Implémenter des factory methods et le pattern Singleton

## Prérequis

- Connaissance des classes et de l'instanciation
- Compréhension de `self` et des attributs d'instance

## 1. `__init__` : l'initialiseur (pas un vrai constructeur)

`__init__` est souvent appelé "constructeur" mais c'est techniquement un **initialiseur**. Il reçoit l'objet déjà créé et l'initialise.

### Caractéristiques de `__init__`

- Appelé automatiquement après la création de l'objet
- Reçoit `self` (l'instance déjà créée) comme premier paramètre
- Ne doit rien retourner (ou retourner `None` implicitement)
- Peut recevoir des paramètres pour initialiser l'objet

In [ ]:
class Utilisateur:
    def __init__(self, nom, email):
        print(f"__init__ appelé pour {nom}")
        print(f"Type de self : {type(self)}")
        print(f"self existe déjà : {self}\n")
        
        self.nom = nom
        self.email = email
        self.actif = True

user = Utilisateur("Alice", "alice@example.com")
print(f"Utilisateur créé : {user.nom}")

## 2. `__new__` : le vrai constructeur

`__new__` est la méthode qui **crée réellement** l'objet. C'est une méthode statique qui retourne une nouvelle instance.

### Séquence de création

1. `__new__` est appelé → crée l'objet
2. `__init__` est appelé → initialise l'objet

`__new__` est rarement utilisé, sauf pour :
- Implémenter des singletons
- Créer des sous-classes de types immuables (int, str, tuple)
- Personnaliser la création d'objets

In [ ]:
class AvecNew:
    def __new__(cls, *args, **kwargs):
        print(f"1. __new__ appelé pour {cls}")
        print(f"   Arguments : args={args}, kwargs={kwargs}")
        
        # Créer l'instance (appel au __new__ de object)
        instance = super().__new__(cls)
        print(f"   Instance créée : {instance}\n")
        return instance
    
    def __init__(self, valeur):
        print(f"2. __init__ appelé")
        print(f"   self existe déjà : {self}")
        self.valeur = valeur
        print(f"   Initialisation terminée\n")

obj = AvecNew(42)
print(f"Résultat final : {obj.valeur}")

In [ ]:
# Exemple : Sous-classer int (type immuable)
class EntierPositif(int):
    """Un entier qui est toujours positif"""
    
    def __new__(cls, valeur):
        # Pour les types immuables, on doit modifier la valeur dans __new__
        # car __init__ est trop tard (l'objet est déjà créé)
        valeur_positive = abs(valeur)
        return super().__new__(cls, valeur_positive)

n1 = EntierPositif(-5)
n2 = EntierPositif(10)

print(f"EntierPositif(-5) = {n1}")  # 5
print(f"EntierPositif(10) = {n2}")  # 10
print(f"Type : {type(n1)}")
print(f"Est un int : {isinstance(n1, int)}")

## 3. `__del__` : le destructeur (finaliseur)

`__del__` est appelé juste avant que l'objet ne soit détruit par le garbage collector.

### ⚠️ Attention

- `__del__` n'est **pas garanti** d'être appelé (ex: arrêt brutal du programme)
- Ne pas l'utiliser pour libérer des ressources critiques (fichiers, connexions)
- Préférer les **context managers** (`with` statement) pour la gestion de ressources
- Peut causer des problèmes avec les références circulaires

In [ ]:
class AvecDestructeur:
    compteur = 0
    
    def __init__(self, nom):
        self.nom = nom
        AvecDestructeur.compteur += 1
        print(f"✅ Création de {self.nom} (total: {AvecDestructeur.compteur})")
    
    def __del__(self):
        AvecDestructeur.compteur -= 1
        print(f"❌ Destruction de {self.nom} (restant: {AvecDestructeur.compteur})")

# Créer des objets
obj1 = AvecDestructeur("Objet 1")
obj2 = AvecDestructeur("Objet 2")

print("\n--- Suppression de obj1 ---")
del obj1  # Supprime la référence, __del__ peut être appelé

print("\n--- Fin du programme (obj2 sera détruit) ---")

## 4. Cycle de vie d'un objet

Le cycle de vie complet d'un objet Python :

```
1. Appel de la classe : MaClasse(args)
   ↓
2. __new__ est appelé → crée l'objet
   ↓
3. __init__ est appelé → initialise l'objet
   ↓
4. Utilisation de l'objet
   ↓
5. Plus de référence vers l'objet
   ↓
6. Garbage Collector marque l'objet
   ↓
7. __del__ est appelé (si défini)
   ↓
8. Mémoire libérée
```

In [ ]:
class CycleVie:
    def __new__(cls, nom):
        print(f"1️⃣ __new__: Création de l'objet pour '{nom}'")
        instance = super().__new__(cls)
        return instance
    
    def __init__(self, nom):
        print(f"2️⃣ __init__: Initialisation de '{nom}'")
        self.nom = nom
    
    def faire_quelque_chose(self):
        print(f"3️⃣ Utilisation: {self.nom} fait quelque chose")
    
    def __del__(self):
        print(f"4️⃣ __del__: Destruction de '{self.nom}'")

print("=== Création ===")
objet = CycleVie("MonObjet")

print("\n=== Utilisation ===")
objet.faire_quelque_chose()

print("\n=== Suppression ===")
del objet

print("\n=== Après suppression ===")

## 5. Garbage Collector et comptage de références

Python utilise deux mécanismes pour gérer la mémoire :

1. **Comptage de références** : chaque objet a un compteur de références
2. **Garbage Collector cyclique** : détecte et supprime les références circulaires

In [ ]:
import sys
import gc

class ObjetTrace:
    def __init__(self, nom):
        self.nom = nom
    
    def __del__(self):
        print(f"Destruction de {self.nom}")

# Créer un objet et vérifier son compteur de références
obj = ObjetTrace("Test")
print(f"Références à obj : {sys.getrefcount(obj) - 1}")  # -1 car getrefcount crée une ref temporaire

# Créer une nouvelle référence
obj2 = obj
print(f"Références après obj2 = obj : {sys.getrefcount(obj) - 1}")

# Créer une autre référence
liste = [obj]
print(f"Références après liste = [obj] : {sys.getrefcount(obj) - 1}")

# Supprimer les références une par une
del obj2
print(f"Références après del obj2 : {sys.getrefcount(obj) - 1}")

del liste
print(f"Références après del liste : {sys.getrefcount(obj) - 1}")

# Supprimer la dernière référence
print("\nSuppression de la dernière référence :")
del obj  # __del__ est appelé immédiatement

In [ ]:
# Garbage Collector pour les références circulaires
class Noeud:
    def __init__(self, nom):
        self.nom = nom
        self.reference = None
    
    def __del__(self):
        print(f"Destruction de {self.nom}")

# Créer une référence circulaire
a = Noeud("A")
b = Noeud("B")
a.reference = b
b.reference = a  # Référence circulaire : A → B → A

print(f"Références à a : {sys.getrefcount(a) - 1}")
print(f"Références à b : {sys.getrefcount(b) - 1}")

# Supprimer les variables
print("\nSuppression des variables a et b :")
del a, b

# Les objets ne sont PAS détruits immédiatement (référence circulaire)
print("Après del a, b (pas encore détruit à cause du cycle)\n")

# Forcer le garbage collector
print("Exécution du garbage collector :")
gc.collect()  # Le GC détecte et supprime le cycle

In [ ]:
# Informations sur le garbage collector
print("Configuration du Garbage Collector :")
print(f"Seuils : {gc.get_threshold()}")
print(f"Compte : {gc.get_count()}")
print(f"GC activé : {gc.isenabled()}")

# Statistiques
collected = gc.collect()
print(f"\nObjets collectés : {collected}")

## 6. Paramètres par défaut dans `__init__`

On peut définir des paramètres par défaut dans `__init__` pour rendre certains paramètres optionnels.

In [ ]:
from datetime import datetime

class Article:
    def __init__(self, titre, auteur, contenu="", date_creation=None, publie=False):
        self.titre = titre
        self.auteur = auteur
        self.contenu = contenu
        # Utiliser la date actuelle si non spécifiée
        self.date_creation = date_creation or datetime.now()
        self.publie = publie
    
    def __repr__(self):
        statut = "Publié" if self.publie else "Brouillon"
        return f"Article('{self.titre}' par {self.auteur}, {statut})"

# Différentes manières de créer un article
article1 = Article("Python OOP", "Alice")
article2 = Article("Design Patterns", "Bob", "Contenu détaillé...", publie=True)
article3 = Article(
    titre="Async Python",
    auteur="Charlie",
    contenu="Introduction à asyncio",
    date_creation=datetime(2024, 1, 1)
)

for article in [article1, article2, article3]:
    print(article)
    print(f"  Créé le : {article.date_creation.strftime('%Y-%m-%d %H:%M:%S')}\n")

⚠️ **Piège** : Ne jamais utiliser de mutable comme valeur par défaut !

In [ ]:
# ❌ MAUVAIS : liste mutable comme défaut
class MauvaiseClasse:
    def __init__(self, items=[]):  # ❌ Piège !
        self.items = items

obj1 = MauvaiseClasse()
obj2 = MauvaiseClasse()

obj1.items.append(1)
obj2.items.append(2)

print(f"obj1.items : {obj1.items}")  # [1, 2] 😱
print(f"obj2.items : {obj2.items}")  # [1, 2] 😱
print(f"Même liste : {obj1.items is obj2.items}")  # True

print("\n" + "="*50 + "\n")

# ✅ CORRECT : None comme défaut
class BonneClasse:
    def __init__(self, items=None):
        self.items = items if items is not None else []  # ✅
        # Ou : self.items = items or []

obj3 = BonneClasse()
obj4 = BonneClasse()

obj3.items.append(1)
obj4.items.append(2)

print(f"obj3.items : {obj3.items}")  # [1] ✅
print(f"obj4.items : {obj4.items}")  # [2] ✅
print(f"Même liste : {obj3.items is obj4.items}")  # False

## 7. Validation dans `__init__`

Il est recommandé de valider les paramètres dans `__init__` pour garantir que l'objet est toujours dans un état cohérent.

In [ ]:
class CompteBancaire:
    def __init__(self, titulaire, solde_initial=0, devise="EUR"):
        # Validation du titulaire
        if not titulaire or not isinstance(titulaire, str):
            raise ValueError("Le titulaire doit être une chaîne non vide")
        
        # Validation du solde
        if not isinstance(solde_initial, (int, float)):
            raise TypeError("Le solde doit être un nombre")
        if solde_initial < 0:
            raise ValueError("Le solde initial ne peut pas être négatif")
        
        # Validation de la devise
        devises_valides = ["EUR", "USD", "GBP", "CHF"]
        if devise not in devises_valides:
            raise ValueError(f"Devise invalide. Choix : {devises_valides}")
        
        # Si toutes les validations passent, on initialise
        self.titulaire = titulaire
        self.solde = solde_initial
        self.devise = devise
    
    def __repr__(self):
        return f"CompteBancaire({self.titulaire}, {self.solde} {self.devise})"

# Tests de validation
try:
    compte1 = CompteBancaire("Alice", 1000)
    print(f"✅ {compte1}")
except ValueError as e:
    print(f"❌ {e}")

try:
    compte2 = CompteBancaire("", 500)  # Titulaire vide
except ValueError as e:
    print(f"❌ Erreur attendue : {e}")

try:
    compte3 = CompteBancaire("Bob", -100)  # Solde négatif
except ValueError as e:
    print(f"❌ Erreur attendue : {e}")

try:
    compte4 = CompteBancaire("Charlie", 200, "JPY")  # Devise invalide
except ValueError as e:
    print(f"❌ Erreur attendue : {e}")

## 8. Pattern : Factory Methods

Au lieu de surcharger `__init__` avec trop de paramètres, on peut utiliser des **méthodes de classe** (factory methods) pour créer des instances de différentes manières.

In [ ]:
from datetime import datetime, timedelta

class Personne:
    def __init__(self, nom, date_naissance):
        self.nom = nom
        self.date_naissance = date_naissance
    
    @classmethod
    def depuis_age(cls, nom, age):
        """Factory method : créer une personne depuis son âge"""
        annee_naissance = datetime.now().year - age
        date_naissance = datetime(annee_naissance, 1, 1)
        return cls(nom, date_naissance)
    
    @classmethod
    def depuis_annee(cls, nom, annee):
        """Factory method : créer une personne depuis son année de naissance"""
        date_naissance = datetime(annee, 1, 1)
        return cls(nom, date_naissance)
    
    def age(self):
        return datetime.now().year - self.date_naissance.year
    
    def __repr__(self):
        return f"Personne('{self.nom}', {self.age()} ans)"

# Différentes manières de créer une personne
p1 = Personne("Alice", datetime(1990, 5, 15))  # Constructeur normal
p2 = Personne.depuis_age("Bob", 30)  # Factory method
p3 = Personne.depuis_annee("Charlie", 1985)  # Factory method

print(p1)
print(p2)
print(p3)

## 9. Pattern : Singleton avec `__new__`

Le pattern Singleton garantit qu'une classe n'a qu'une seule instance. On utilise `__new__` pour contrôler la création.

In [ ]:
class Singleton:
    _instance = None  # Attribut de classe pour stocker l'unique instance
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            print("Création de l'unique instance")
            cls._instance = super().__new__(cls)
        else:
            print("Retour de l'instance existante")
        return cls._instance
    
    def __init__(self, valeur):
        # ⚠️ __init__ est appelé à chaque fois !
        print(f"__init__ appelé avec valeur={valeur}")
        self.valeur = valeur

# Créer plusieurs "instances"
s1 = Singleton(1)
print(f"s1.valeur = {s1.valeur}\n")

s2 = Singleton(2)
print(f"s2.valeur = {s2.valeur}")
print(f"s1.valeur = {s1.valeur}  (modifié !)\n")

# Vérifier que c'est la même instance
print(f"s1 is s2 : {s1 is s2}")
print(f"id(s1) : {id(s1)}")
print(f"id(s2) : {id(s2)}")

In [ ]:
# Version améliorée : éviter de réinitialiser
class SingletonAmeliore:
    _instance = None
    _initialized = False
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, valeur):
        if not SingletonAmeliore._initialized:
            print(f"Initialisation avec valeur={valeur}")
            self.valeur = valeur
            SingletonAmeliore._initialized = True
        else:
            print(f"Déjà initialisé, valeur={valeur} ignorée")

s3 = SingletonAmeliore(10)
print(f"s3.valeur = {s3.valeur}\n")

s4 = SingletonAmeliore(20)
print(f"s4.valeur = {s4.valeur}")
print(f"s3.valeur = {s3.valeur}  (inchangé !)")

## Pièges courants

### Piège 1 : `__del__` n'est pas garanti d'être appelé

In [ ]:
# ❌ MAUVAIS : Utiliser __del__ pour fermer un fichier
class MauvaisGestionnaireFichier:
    def __init__(self, nom_fichier):
        self.fichier = open(nom_fichier, 'w')
    
    def __del__(self):
        # Peut ne jamais être appelé !
        self.fichier.close()

# ✅ BON : Utiliser un context manager
class BonGestionnaireFichier:
    def __init__(self, nom_fichier):
        self.nom_fichier = nom_fichier
        self.fichier = None
    
    def __enter__(self):
        self.fichier = open(self.nom_fichier, 'w')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.fichier:
            self.fichier.close()
            print("Fichier fermé proprement")

# Utilisation avec 'with' garantit la fermeture
with BonGestionnaireFichier('/tmp/test.txt') as gf:
    gf.fichier.write("Contenu\n")
# Le fichier est fermé automatiquement ici

### Piège 2 : Références circulaires avec `__del__`

In [ ]:
# Références circulaires peuvent empêcher __del__ d'être appelé
class NoeudProblematique:
    def __init__(self, nom):
        self.nom = nom
        self.enfants = []
    
    def __del__(self):
        print(f"Destruction de {self.nom}")

# Créer un cycle
parent = NoeudProblematique("Parent")
enfant = NoeudProblematique("Enfant")
parent.enfants.append(enfant)
enfant.parent = parent  # Cycle

print("Suppression des variables...")
del parent, enfant
print("Variables supprimées\n")

# Force le GC
print("Garbage collection...")
gc.collect()
print("Done")

### Piège 3 : `__new__` rarement nécessaire

In [ ]:
# ❌ Utiliser __new__ quand __init__ suffit
class InutilementComplexe:
    def __new__(cls, valeur):
        instance = super().__new__(cls)
        # ❌ Mauvaise pratique : initialiser dans __new__
        instance.valeur = valeur * 2
        return instance
    
    def __init__(self, valeur):
        # Confusion : valeur déjà modifiée dans __new__
        pass

# ✅ Simplement utiliser __init__
class Simple:
    def __init__(self, valeur):
        self.valeur = valeur * 2

obj = Simple(5)
print(f"Valeur : {obj.valeur}")

## Mini-exercices

### Exercice 1 : Classe avec validation

Créez une classe `Rectangle` avec :
- Attributs : `largeur`, `hauteur`
- Validation dans `__init__` : les dimensions doivent être > 0
- Méthode `aire()` qui retourne l'aire
- Méthode `perimetre()` qui retourne le périmètre

Testez avec des valeurs valides et invalides.

In [ ]:
# Votre code ici


### Solution Exercice 1

In [ ]:
class Rectangle:
    def __init__(self, largeur, hauteur):
        # Validation
        if not isinstance(largeur, (int, float)) or not isinstance(hauteur, (int, float)):
            raise TypeError("Les dimensions doivent être des nombres")
        if largeur <= 0 or hauteur <= 0:
            raise ValueError("Les dimensions doivent être strictement positives")
        
        self.largeur = largeur
        self.hauteur = hauteur
    
    def aire(self):
        return self.largeur * self.hauteur
    
    def perimetre(self):
        return 2 * (self.largeur + self.hauteur)
    
    def __repr__(self):
        return f"Rectangle({self.largeur}x{self.hauteur})"

# Tests valides
r1 = Rectangle(10, 5)
print(f"{r1} - Aire: {r1.aire()}, Périmètre: {r1.perimetre()}")

# Tests invalides
try:
    r2 = Rectangle(-5, 10)
except ValueError as e:
    print(f"❌ Erreur attendue : {e}")

try:
    r3 = Rectangle("abc", 10)
except TypeError as e:
    print(f"❌ Erreur attendue : {e}")

### Exercice 2 : Singleton avec compteur

Créez une classe `Compteur` qui est un Singleton avec :
- Un attribut `valeur` initialisé à 0
- Méthode `incrementer()` qui incrémente la valeur
- Méthode `get_valeur()` qui retourne la valeur

Vérifiez que plusieurs "instances" partagent le même compteur.

In [ ]:
# Votre code ici


### Solution Exercice 2

In [ ]:
class Compteur:
    _instance = None
    _initialized = False
    
    def __new__(cls):
        if cls._instance is None:
            print("Création du Singleton Compteur")
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not Compteur._initialized:
            self.valeur = 0
            Compteur._initialized = True
    
    def incrementer(self):
        self.valeur += 1
    
    def get_valeur(self):
        return self.valeur

# Tests
c1 = Compteur()
c2 = Compteur()

print(f"c1 is c2 : {c1 is c2}")

c1.incrementer()
c1.incrementer()
print(f"c1.get_valeur() : {c1.get_valeur()}")
print(f"c2.get_valeur() : {c2.get_valeur()}")  # Même valeur

c2.incrementer()
print(f"c1.get_valeur() : {c1.get_valeur()}")  # Incrémenté aussi
print(f"c2.get_valeur() : {c2.get_valeur()}")

### Exercice 3 : Observer le Garbage Collector

Créez une classe `ObjetTrace` qui :
- Affiche un message dans `__init__` et `__del__`
- A un attribut de classe `compteur` qui compte les instances vivantes

Créez plusieurs objets, supprimez-en certains, et observez quand `__del__` est appelé.

In [ ]:
# Votre code ici


### Solution Exercice 3

In [ ]:
class ObjetTrace:
    compteur = 0
    
    def __init__(self, nom):
        self.nom = nom
        ObjetTrace.compteur += 1
        print(f"✅ Création de '{self.nom}' (total vivants: {ObjetTrace.compteur})")
    
    def __del__(self):
        ObjetTrace.compteur -= 1
        print(f"❌ Destruction de '{self.nom}' (total vivants: {ObjetTrace.compteur})")

print("=== Création de 3 objets ===")
obj1 = ObjetTrace("A")
obj2 = ObjetTrace("B")
obj3 = ObjetTrace("C")

print(f"\nObjets vivants : {ObjetTrace.compteur}")

print("\n=== Suppression de obj2 ===")
del obj2

print(f"\nObjets vivants : {ObjetTrace.compteur}")

print("\n=== Création d'une liste avec obj1 ===")
ma_liste = [obj1]
print(f"Références à obj1 : {sys.getrefcount(obj1) - 1}")

print("\n=== Suppression de obj1 (mais référence dans liste) ===")
del obj1  # Pas encore détruit car dans ma_liste
print("obj1 toujours vivant (dans ma_liste)")

print("\n=== Suppression de la liste ===")
del ma_liste  # Maintenant obj1 est détruit

print(f"\nObjets vivants : {ObjetTrace.compteur}")

print("\n=== Fin du programme (obj3 sera détruit) ===")

## Résumé

Dans ce notebook, vous avez appris :

- ✅ `__init__` est un initialiseur (pas un vrai constructeur)
- ✅ `__new__` est le vrai constructeur qui crée l'objet
- ✅ `__del__` est un destructeur/finaliseur (non garanti)
- ✅ Le cycle de vie : `__new__` → `__init__` → utilisation → `__del__` → libération
- ✅ Le garbage collector utilise le comptage de références et détecte les cycles
- ✅ Paramètres par défaut dans `__init__` (attention aux mutables !)
- ✅ Validation des paramètres dans `__init__`
- ✅ Factory methods avec `@classmethod`
- ✅ Pattern Singleton avec `__new__`
- ✅ Préférer les context managers à `__del__` pour les ressources

**Prochaine étape** : Encapsulation et visibilité (`_`, `__`, `@property`)